# My turn

In [24]:
import numpy as np
import random
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline

In [25]:
pb_data = np.loadtxt('mass_atten_pb.dat', delimiter=' ')
pb_energy = pb_data[:, 0]
pb_scatter = pb_data[:, 1]
pb_absorb = pb_data[:, 2]
pb_pair_production = pb_data[:, 3]
pb_total_XS = pb_scatter + pb_absorb + pb_pair_production
pb_E_depo = pb_data[:, 6]

h2o_data = np.loadtxt('mass_atten_h2o.dat', delimiter=' ')
h2o_energy = pb_data[:, 0]
h2o_scatter = pb_data[:, 1]
h2o_absorb = pb_data[:, 2]
h2o_pair_production = pb_data[:, 3]
h2o_total_XS = pb_scatter + pb_absorb + pb_pair_production
h2o_E_depo = pb_data[:, 6]

In [26]:
def interpolate2(E, arr_E: np.ndarray, arr_inter: np.ndarray):
    i = len(arr_E[arr_E < E])
    
    log_x1 = np.log(arr_E[i])
    log_x2 = np.log(arr_E[i+1])
    log_x = np.log(E)
    
    log_y1 = np.log(arr_inter[i])
    log_y2 = np.log(arr_inter[i+1])
    
    log_y = log_y1 + (log_x-log_x1) * (log_y2 - log_y1) / (log_x2 - log_x1)
    y = np.exp(log_y)

    return y

In [27]:
def interpolate(E, arr_E: np.ndarray, arr_inter: np.ndarray):
    return np.exp(np.interp(
    np.log(E),
    np.log(arr_E),
    np.log(arr_inter)
        ))

In [28]:
# My particle gambling functions
def r_sample(radius):
    r = radius* ((np.random.random())**(1/3))
    return r
def alpha_sample():
    return 2*np.pi*np.random.random()
def omega_sample():
    return 2*np.random.random() - 1

def mu_sample():
    return 2*np.random.random() - 1
def gamma_sample():
    return 2*np.pi*np.random.random()
    
def x_sample(radius, alpha, omega):
    return radius*np.cos(alpha)*np.sqrt(1 - (omega**2))
def y_sample(radius, alpha, omega):
    return radius*np.sin(alpha)*np.sqrt(1 - (omega**2))
def z_sample(radius, omega):
    return radius*omega
    

In [29]:
# Alright enough lollie-gagging (I think that's how you spell it). Time to code my MC-Particle Simulation
rng = np.random.random()

class MyParticle:
    energy = 0
    d_energy = 0
    
    x = 0
    y = 0
    z = 0
    
    omega_x = 0
    omega_y = 0
    omega_z = 0
    
    alive = False  
    
    def __init__(self, x, y, z, energy, d_energy): 
        self.energy = energy
        self.d_energy = d_energy
        
        self.x = x
        self.y = y
        self.z = z
        
        mu = mu_sample()
        gamma = gamma_sample()
        
        self.omega_x = np.sqrt(1 - mu**2)*np.cos(gamma)
        self.omega_y = np.sqrt(1 - mu**2)*np.sin(gamma)
        self.omega_z = mu
        
        self.alive = True
    
    def print_vals(self):
        print('Here are the values of this particle')
        print(f"The energy is: {self.energy}")
        print(f"The x position is: {self.x}")
        print(f"The y position is: {self.y}")
        print(f"The z position is: {self.z}") 
    
    def make_dead(self):
        self.alive = False


In [30]:
class Sphere:
    radius = 0
    E_arr = 0
    E_t = 0
    E_s = 0
    E_p = 0
    
    def __init__(self, radius, E_arr:np.ndarray, E_t:np.ndarray, E_s:np.ndarray, E_p:np.ndarray):
        self.radius = radius
        self.E_arr = E_arr
        self.E_t = E_t
        self.E_p = E_p
        self.E_s = E_s
        
    
    def sample_particle(self, energy, d_energy=None):
        r = r_sample(self.radius)
        w = 2*np.random.random() - 1
        a = 2*np.pi*np.random.random()
        
        x = x_sample(r, a, w)
        y = y_sample(r, a, w)
        z = z_sample(r, w)
        
        return MyParticle(x, y, z, energy, d_energy) 
    
    # Distance determination functions -----------------

    # Distance to sphere surface
    
    def surf_dist(self, particle:MyParticle):
        b = 2*(particle.x*particle.omega_x + particle.y*particle.omega_y + particle.z*particle.omega_z)
        c = particle.x**2 + particle.y**2 + particle.z**2 - self.radius**2
        
        pos = (-b + np.sqrt(b**2 - 4*c))*0.5
        if pos > 0:
            return pos
            
        else:
            return (-b - np.sqrt(b**2 - 4*c))*0.5
        
    # Distance to next collision assuming
    def coll_dist_abs(self, particle:MyParticle):
        E_t = interpolate2(particle.energy, self.E_arr, self.E_t) 
        E_t_d = interpolate2(particle.energy + particle.d_energy, self.E_arr, self.E_t)   
         
        d = self.surf_dist(particle)
        weight = np.exp(-(E_t_d - E_t) * d)
        return -np.log(np.random.random())/E_t, weight
    
    def is_in_sphere(self, particle:MyParticle, x = 0, y = 0, z = 0):
     if (x - particle.x)**2 + (y - particle.y)**2 + (z - particle.z)**2 <= self.radius**2:
        return True
     else:
         return False
    
    
    def compton_scatter(self, particle:MyParticle, dist):
        particle.x =  (particle.x + dist*particle.omega_x)
        particle.y =  (particle.y + dist*particle.omega_y)
        particle.z =  (particle.z + dist*particle.omega_z)
    
        a = particle.energy/0.511
        
        A_inv = (2*a+2)/(4*(a**2)+4*a+1) + np.log(2*a+1)/(a) - 2/(2*a+1) - ((4*(a**2)+6*a+2)*np.log(2*a+1)-2*(a**3)-8*(a**2)-4*a)/(2*(a**4)+(a**3))
        A = 1 / A_inv

        def kappa(mu):
            kappa = 1 / (1 + a * (1-mu))
            return kappa
        def p_s(mu):
            k = kappa(mu)
            p = A * k**2 * (k + (1/k) - (1-mu**2))
            return p

        p0 = p_s(-1)
        p1 = p_s( 1)
        r = p1 / p0
        c = 2*p0 * (r-1) / np.log(r)

        def g(mu):
            g = (r ** ((mu+1)/2)) * p0 / c
            return g
        def calc_mu(xi): #The inverse of g
            mu = (2*np.log(c*xi*np.log(r)/(2*p0) + 1)  -  np.log(r)) / np.log(r)
            return mu

        mu_0 = 0
        gamma_0 = 2 * np.pi * random.random()

        still_looking = True
        while still_looking:
            x_hat = calc_mu(random.random())
            y_hat = random.random() * c * g(x_hat)
            if y_hat < g(x_hat):
                mu_0 = x_hat
                still_looking = False

        E_ratio = kappa(mu_0)
        
        new_particle = particle
        
        new_particle.energy = particle.energy*E_ratio
        
        new_particle.omega_x = (particle.omega_x*mu_0 + 
                                np.sqrt(1 - mu_0**2)*(particle.omega_x*particle.omega_y*np.cos(gamma_0) - 
                                                            particle.omega_y*np.sin(gamma_0)))/np.sqrt(1-particle.omega_z**2)
        
        new_particle.omega_y = (particle.omega_y*mu_0 + 
                                np.sqrt(1 - mu_0**2)*(particle.omega_y*particle.omega_z*np.cos(gamma_0) + 
                                                            particle.omega_x*np.sin(gamma_0)))/np.sqrt(1-particle.omega_z**2)

        if(new_particle.omega_z == 1):
            print('fuck')
        
        new_particle.omega_z = (particle.omega_z*mu_0 - np.sqrt(1 - mu_0**2)*np.sqrt(1 - (particle.omega_z)**2)*np.cos(gamma_0))
        
        norma_const = 1/np.sqrt(new_particle.omega_x**2 + new_particle.omega_y**2 + new_particle.omega_z**2)
        
        new_particle.omega_x *= norma_const
        new_particle.omega_y *= norma_const
        new_particle.omega_z *= norma_const

        return new_particle
        
    def pair_production(self, old_particle:MyParticle):
        base_E = 0.511
        
        photon_1 = self.sample_particle(base_E, old_particle.d_energy)
        photon_2 = self.sample_particle(base_E, old_particle.d_energy)
        
        photon_1.x = old_particle.x
        photon_1.y = old_particle.y
        photon_1.z = old_particle.z
        
        photon_2.x = old_particle.x
        photon_2.y = old_particle.y
        photon_2.z = old_particle.z
        
        photon_2.omega_x = -photon_1.omega_x
        photon_2.omega_y = -photon_1.omega_y
        photon_2.omega_z = -photon_1.omega_z    
        
        return photon_1, photon_2

In [33]:
def history(histories, energy, dE, r, E_arr:np.ndarray ,E_t:np.ndarray, E_s:np.ndarray, E_p:np.ndarray):
    leaked = 0
    dleaked = 0
    sphere = Sphere(r, E_arr, E_t, E_s, E_p)
    
    particle_bank = np.array([sphere.sample_particle(energy, dE) for _ in range(int(histories*5))], dtype=object)
    weights = np.array([0 for _ in range(int(histories*5))], dtype=float)
    #particle_bank = np.array([sphere.sample_particle(energy) for _ in range(int(histories*5))], dtype=object)
    bank_tracker = histories
    i = 0
    
    #while i < bank_tracker:
    #while i < len(particle_bank):   
    while i < bank_tracker:         
        while particle_bank[i].alive == True:
            E_t_current = interpolate2(particle_bank[i].energy, E_arr, E_t)
            E_s_current = interpolate2(particle_bank[i].energy, E_arr, E_s)
            E_p_current = interpolate2(particle_bank[i].energy, E_arr, E_p)
            
            l_s = sphere.surf_dist(particle_bank[i])
            l_c, w = sphere.coll_dist_abs(particle_bank[i])
            weights[i] = w
            
            if l_s < l_c:
                particle_bank[i].make_dead()
                leaked += 1
                dleaked += 1*w
            else:
                xi = random.random()
                if xi < E_s_current/E_t_current:
                    particle_bank[i] = sphere.compton_scatter(particle_bank[i], l_c)
                elif xi < (E_s_current + E_p_current)/E_t_current:
                    p1, p2 = sphere.pair_production(particle_bank[i])
                    particle_bank[i].make_dead()
                    
                    #particle_bank = np.append(particle_bank, p1)
                    #particle_bank = np.append(particle_bank, p2)  
                    particle_bank[int(bank_tracker)] = p1
                    particle_bank[int(bank_tracker)+1] = p2
                    bank_tracker += 2
                else:
                    particle_bank[i].make_dead()
        
        i += 1
        
    return leaked, dleaked, weights



In [34]:
# Lets generate a monoenergetic sample of particles
# I am thinking 1E6 neutrons with an energy of 1 MeV scattered about a sphere of radius 5 units
starting_energy = 4.5 # MeV
dE = -2 
sphere_radius = 5
histories = 1E5
p_pb = 11.35 # g/cm3
E_t = pb_total_XS*p_pb
E_s = pb_scatter*p_pb
E_p = pb_pair_production*p_pb
w = []

leaked, dleaked, w = history(histories, starting_energy, dE, sphere_radius, pb_energy, E_t, E_s, E_p)
    
print(f"In the purely absorbing sphere of radius {sphere_radius}, \n {leaked} particles were leaked out of {histories}")
print(f"Leakage rate for an energy of {starting_energy} is {leaked/histories}")
print(f"Leakage rate for an energy of {starting_energy + dE} is {dleaked/histories}")

/var/folders/nz/tycrf9m51dd85jb0tgdyntqc0000gn/T/ipykernel_48404/897680474.py:8: RuntimeWarning: divide by zero encountered in log
  log_y1 = np.log(arr_inter[i])
/var/folders/nz/tycrf9m51dd85jb0tgdyntqc0000gn/T/ipykernel_48404/897680474.py:9: RuntimeWarning: divide by zero encountered in log
  log_y2 = np.log(arr_inter[i+1])
/var/folders/nz/tycrf9m51dd85jb0tgdyntqc0000gn/T/ipykernel_48404/897680474.py:11: RuntimeWarning: invalid value encountered in scalar subtract
  log_y = log_y1 + (log_x-log_x1) * (log_y2 - log_y1) / (log_x2 - log_x1)
/var/folders/nz/tycrf9m51dd85jb0tgdyntqc0000gn/T/ipykernel_48404/897680474.py:6: RuntimeWarning: invalid value encountered in log
  log_x = np.log(E)
/var/folders/nz/tycrf9m51dd85jb0tgdyntqc0000gn/T/ipykernel_48404/2768969302.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  return -np.log(np.random.random())/E_t, weight


In the purely absorbing sphere of radius 5, 
 44703 particles were leaked out of 100000.0
Leakage rate for an energy of 4.5 is 0.44703
Leakage rate for an energy of 2.5 is nan
